<a href="https://colab.research.google.com/github/VenkteshV/Question_duplicate_detection/blob/main/Week9/(Maksim)Extracting_Keywords_using_diff_models_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet keybert
!pip install --quiet git+https://github.com/boudinfl/pke.git

     |████████████████████████████████| 85 kB 4.3 MB/s 
     |████████████████████████████████| 235 kB 63.7 MB/s 
     |████████████████████████████████| 51 kB 6.9 MB/s 
     |████████████████████████████████| 4.4 MB 57.8 MB/s 
     |████████████████████████████████| 1.2 MB 54.3 MB/s 
     |████████████████████████████████| 101 kB 12.2 MB/s 
     |████████████████████████████████| 596 kB 52.6 MB/s 
     |████████████████████████████████| 6.6 MB 61.1 MB/s 


In [ ]:
import pandas as pd
import spacy
import pke
from tqdm import tqdm
from spacy.matcher import Matcher
import time
import numpy as np
from keybert import KeyBERT

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Extramarks Project/Dataset versions/Model1-Semantic/Jacc_sim_ignore_syllabus.csv")
df = df.drop("Unnamed: 0", axis= 1)
df = df[:100]
df

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
pke_extractors = [
                  # pke.unsupervised.TfIdf, 
                  # pke.unsupervised.YAKE, 
                  # pke.unsupervised.TextRank,  
                  # pke.unsupervised.TopicRank, 
                  pke.unsupervised.TopicalPageRank, 
                  pke.unsupervised.PositionRank, 
                  pke.unsupervised.MultipartiteRank
                  ]

In [ ]:
def keyword_model_extractor(ind, text):
    extractor = pke_extractors[ind]()
    extractor.load_document(text, language='en')
    pos = {'NOUN', 'PROPN', 'ADJ', 'ADV'}

    # if pke_extractors[ind].__name__ == "TfIdf":
    #   extractor.candidate_selection(n=3)
    if pke_extractors[ind].__name__ == "KPMiner":
      extractor.candidate_selections()
    if pke_extractors[ind].__name__ == "YAKE":
      extractor.candidate_selection()
    elif pke_extractors[ind].__name__ == "TextRank":
      extractor.candidate_selection(pos = pos)
    elif pke_extractors[ind].__name__ == "TopicRank":
      extractor.candidate_selection(pos = pos)
    elif pke_extractors[ind].__name__ == "TopicalPageRank":
      extractor.candidate_selection()
    elif pke_extractors[ind].__name__ == "PositionRank":
      extractor.candidate_selection()
    elif pke_extractors[ind].__name__ == "MultipartiteRank":
      extractor.candidate_selection(pos = pos)


    extractor.candidate_weighting()
    keyphrases = extractor.get_n_best(n=10)
    results = []
    for scored_keywords in keyphrases:
        for keyword in scored_keywords:
            if isinstance(keyword, str):
                results.append(keyword) 
    return results

In [ ]:
# for ind in range(len(pke_extractors)):
#   print("Model name : "+  pke_extractors[ind].__name__)
#   corpus_kws = {}
#   try:
#     for idx, text in tqdm(enumerate(questions_ls)):
#       corpus_kws[idx] = keyword_model_extractor(ind, text)
#   except:
#     corpus_kws[idx] = []
#   corpus_kws_df = pd.DataFrame(pd.Series(corpus_kws), columns=["keywords"])
#   pd.concat([df, corpus_kws_df], axis=1).to_csv('/content/drive/MyDrive/Extramarks Project/Week8/'+ pke_extractors[ind].__name__+ '-extracted_keywords.csv')

In [ ]:
def intersection_list(list1, list2):  
   list3 = [value for value in list1 if value in list2]  
   return list3 

def union_list(list1, list2):
  set1 = set(list1)
  set2 = set(list2)
  newList = list(set1.union(set2)) 
  return newList

In [ ]:
def keyword_score(question1, question2, ind):

  # print(question1)
  # print(question2)

  try :
    keyword1 = keyword_model_extractor(ind, question1)
  except :
    keyword1 = []

  try :
    keyword2 = keyword_model_extractor(ind, question2)
  except :
    keyword2 = []

  common_keyword = intersection_list(keyword1, keyword2)
  union_keyword = union_list(keyword1, keyword2)

  if(union_keyword == []):      # suggestive model
    return 1

  return len(common_keyword)/len(union_keyword)


In [ ]:
# df = df.reset_index()  # make sure indexes pair with number of rows

# 1. Split keywords
# 2. Embeddings

threshold_val = 0.5
score_values = []
for index, row in df.iterrows():
    if index % 250 == 0:
      print(index)
    question1 = row['ques1'] 
    question2 = row['ques2']
    score_values.append(keyword_score(question1, question2, 0))

In [ ]:
new_df = pd.DataFrame(df)
new_df['keword_score'] = score_values

In [ ]:
new_df

In [ ]:
new_df.to_csv('/content/drive/MyDrive/Extramarks Project/Dataset versions/Model2-Keyword/Added_keyword_overlap_score.csv', encoding='utf-8', index=False)